#What to eat?

In [ ]:
# Install necessary Python packages - these are libraries that add extra functionality
!pip install --upgrade openai requests aiohttp nest_asyncio gradio

# Import required libraries
import openai              # Used to communicate with AI models like ChatGPT
import json                # Helps work with JSON data (a common web data format)
import urllib.parse        # Helps format text for web addresses
import requests            # Allows the code to make web requests (like visiting websites)
import asyncio             # Helps run multiple tasks at the same time
import aiohttp             # Similar to requests but works with asyncio
import time                # Provides time-related functions
import logging             # Helps track what the program is doing
from functools import lru_cache  # Saves results of functions to avoid repeating work
import nest_asyncio        # Makes asyncio work in environments like Jupyter notebooks
import gradio as gr        # Creates simple web interfaces
from IPython.core.display import display, HTML  # Displays content in Jupyter notebooks
# Allow nested asyncio loops - technical setting needed for Jupyter/Colab
nest_asyncio.apply()

# Set up logging to track what's happening while the program runs
logging.basicConfig(level=logging.INFO)


In [ ]:
# --- SET UP API KEYS ---
# You'll need to replace these with your own API keys
client = openai.OpenAI(api_key="YOUR_OPENAI_API_KEY")  # Connect to OpenAI (ChatGPT)
GOOGLE_MAPS_API_KEY = "YOUR_GOOGLE_MAPS_API_KEY"       # Connect to Google Maps

In [ ]:
# --- LOCATION FINDER FUNCTION ---
# This function is "cached" - it remembers previous answers to avoid unnecessary work
@lru_cache(maxsize=32)
def geocode_location(location, maps_api_key):
    """Turn a location name into GPS coordinates (latitude and longitude)."""
    # Create a URL to ask Google Maps where this location is
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={urllib.parse.quote(location)}&key={maps_api_key}"

    try:
        # Send the request and get the response
        resp = requests.get(geocode_url)
        data = resp.json()  # Convert the response to JSON format

        # If Google found the location successfully
        if data["status"] == "OK":
            # Extract the latitude and longitude
            loc = data["results"][0]["geometry"]["location"]
            return loc["lat"], loc["lng"]
        else:
            # If there was an error, log it and return None
            logging.error(f"Geocode error for '{location}': {data.get('status')}")
            return None, None
    except Exception as e:
        # If something unexpected happened, log the error
        logging.error(f"Exception during geocoding: {e}")
        return None, None


In [ ]:
# --- RESTAURANT SEARCH FUNCTION ---
def get_nearby_restaurants(location, maps_api_key):
    """Find restaurants within 1km of a location."""
    # First, convert the location name to coordinates
    lat, lng = geocode_location(location, maps_api_key)
    if lat is None or lng is None:
        return []  # Return empty list if we couldn't find the location

    # Create the URL for searching restaurants nearby
    base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=1000&type=restaurant&key={maps_api_key}"
    results = []
    next_page_token = None

    # Google might return multiple pages of results, so keep going until we get them all
    while True:
        # If this is a follow-up page, add the page token
        url = base_url if not next_page_token else f"{base_url}&pagetoken={next_page_token}"

        try:
            # Get the results for this page
            resp = requests.get(url)
            data = resp.json()

            # Check if the request was successful
            if data.get("status") not in ["OK", "ZERO_RESULTS"]:
                logging.error(f"Nearby search error: {data.get('status')}")
                break

            # Process each restaurant found
            for place in data.get("results", []):
                results.append({
                    "name": place.get("name", "Unknown"),           # Restaurant name
                    "address": place.get("vicinity", "Unknown"),    # Address
                    "place_id": place.get("place_id", None),        # Unique ID for this place
                    "rating": place.get("rating", "N/A"),           # Star rating
                    "reviews": place.get("user_ratings_total", "N/A"), # Number of reviews
                    "geometry": place.get("geometry", {})           # Location data
                })

            # Check if there are more pages of results
            next_page_token = data.get("next_page_token")
            if not next_page_token:
                break

            # Wait before requesting the next page (API requirement)
            time.sleep(2)

        except Exception as e:
            # Log any errors
            logging.error(f"Exception during Nearby Search: {e}")
            break

    return results

In [ ]:
# --- DETAILED RESTAURANT INFO FUNCTION (ASYNC VERSION) ---
# This function gets more details about a restaurant and runs in the background
async def async_get_restaurant_details(session, place_id, maps_api_key):
    """Get detailed information about a specific restaurant."""
    # Create URL to get restaurant details
    url = (f"https://maps.googleapis.com/maps/api/place/details/json?"
           f"place_id={place_id}&fields=reviews,rating,user_ratings_total,formatted_address,name&key={maps_api_key}")

    try:
        # Get the detailed information
        async with session.get(url) as resp:
            data = await resp.json()

            # Set up default values in case some data is missing
            details = {
                "name": "Unknown",
                "address": "Unknown",
                "rating": "N/A",
                "reviews": "N/A",
                "review_summary": "No reviews found.",
                "place_id": place_id
            }

            # Extract the data if it exists
            if "result" in data:
                details["name"] = data["result"].get("name", "Unknown")
                details["address"] = data["result"].get("formatted_address", "Unknown")
                details["rating"] = data["result"].get("rating", "N/A")
                details["reviews"] = data["result"].get("user_ratings_total", "N/A")

                # If there are reviews, get the newest ones and analyze them
                if "reviews" in data["result"]:
                    sorted_reviews = sorted(data["result"]["reviews"], key=lambda x: x.get("time", 0), reverse=True)
                    newest_reviews = [rev.get("text", "") for rev in sorted_reviews[:5]]
                    if newest_reviews:
                        details["review_summary"] = analyze_reviews(newest_reviews)

            return details

    except Exception as e:
        # Log any errors
        logging.error(f"Error fetching details for place_id {place_id}: {e}")
        return {"place_id": place_id}

In [ ]:
# This function gets details for multiple restaurants at once
async def fetch_all_restaurant_details(place_ids, maps_api_key):
    """Get details for multiple restaurants in parallel."""
    async with aiohttp.ClientSession() as session:
        # Create a task for each restaurant
        tasks = [async_get_restaurant_details(session, pid, maps_api_key) for pid in place_ids]
        # Run all tasks and wait for them to complete
        return await asyncio.gather(*tasks)

In [ ]:
# --- REVIEW ANALYSIS FUNCTION ---
def analyze_reviews(reviews_list):
    """Use ChatGPT to analyze and summarize restaurant reviews."""
    # Combine all reviews into one text
    reviews_text = "\n".join(reviews_list)

    # Create a prompt for ChatGPT
    prompt = (
        "Analyze the following reviews for a restaurant and provide a concise summary "
        "that captures the overall sentiment and key points:\n\n"
        f"Reviews:\n{reviews_text}\n\nSummary:"
    )

    try:
        # Ask ChatGPT to analyze the reviews
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7  # Controls randomness: higher = more creative
        )
        # Get the summary from ChatGPT's response
        summary = response.choices[0].message.content.strip()
        return summary

    except Exception as e:
        # Log any errors
        logging.error(f"Error in ChatGPT review analysis: {e}")
        return "Review analysis failed."

In [ ]:
# --- MAP EMBEDDING FUNCTION ---
def get_map_embed_html(place_id, maps_api_key, latitude=None, longitude=None):
    """Create HTML code to show a Google Map for a restaurant."""
    # If we have a place ID, use that (more accurate)
    if place_id:
        embed_url = f"https://www.google.com/maps/embed/v1/place?key={maps_api_key}&q=place_id:{place_id}"
    # Otherwise use the coordinates if available
    elif latitude and longitude:
        embed_url = f"https://www.google.com/maps/embed/v1/view?key={maps_api_key}&center={latitude},{longitude}&zoom=15"
    # If neither is available, show an error message
    else:
        return "<p>No map available.</p>"

    # Return an HTML iframe that shows the map
    return f'<iframe width="100%" height="400" frameborder="0" style="border:0" src="{embed_url}" allowfullscreen></iframe>'


In [ ]:
# --- RESTAURANT RECOMMENDATION FUNCTION ---
def choose_best_restaurant(restaurants, people_details, location, price_range, ambience, food_requirements):
    """Use ChatGPT to choose the best restaurant for a group of people."""
    # Start building a prompt for ChatGPT
    prompt = f"Given the following list of restaurants near {location} in Singapore and these group details:\n"

    # Add each person's requirements
    for detail in people_details:
        prompt += f"- {detail}\n"

    # Add other preferences
    prompt += f"Price Range: {price_range or 'Any'}\n"
    prompt += f"Ambience: {ambience or 'Any'}\n"
    prompt += f"Food Requirements: {food_requirements or 'None'}\n\n"

    # Add instructions for ChatGPT
    prompt += ("For each restaurant below, note its name, address, rating, total reviews, and review summary. "
               "Then, choose the single best restaurant that best meets the group's needs that do not have serious negative reviews, with higher priority on fulfilling each individual's specific requirements. "
               "In your explanation, you MUST clearly state how the restaurant meets every individual's needs cannot just group them together and say it is good for the group. "
               "If any requirement cannot be met, explain why a compromise was necessary and include an apology for that compromise. "
               "Use the restaurant's number (e.g., '1', '2', etc.) in your response.\n\n")

    # Add information about each restaurant
    for i, r in enumerate(restaurants):
        prompt += (f"{i+1}. {r['name']} ({r['address']}) - Rating: {r['rating']}, Reviews: {r['reviews']}\n"
                   f"   Review Summary: {r.get('review_summary', 'No summary available.')}\n\n")

    # Ask for a structured response
    prompt += "Output JSON with keys 'chosen_restaurant_index' and 'reason':"

    try:
        # Ask ChatGPT to choose the best restaurant
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )

        # Extract and process the response
        content = response.choices[0].message.content.strip()
        chosen = json.loads(content)  # Convert the JSON response to a Python object

        # Get the chosen restaurant index (subtract 1 because Python uses 0-based indexing)
        index = int(chosen.get("chosen_restaurant_index")) - 1

        # Make sure the index is valid
        if index < 0 or index >= len(restaurants):
            logging.error("Chosen index out of range. Defaulting to the first restaurant.")
            index = 0

        # Get the reasoning
        reason = chosen.get("reason", "No reasoning provided.")

        # Return the chosen restaurant and the reasoning
        return restaurants[index], reason

    except Exception as e:
        # Log any errors
        logging.error(f"Error in choosing best restaurant: {e}")
        return None, f"Error: {e}"

In [ ]:
# --- HTML TABLE GENERATOR ---
def generate_restaurants_table(restaurants):
    """Create an HTML table showing all restaurant details."""
    # Start the HTML table
    html = "<table border='1' style='border-collapse: collapse; width: 100%;'>"

    # Add table headers
    html += ("<tr>"
             "<th>#</th>"
             "<th>Name</th>"
             "<th>Address</th>"
             "<th>Rating</th>"
             "<th>Total Reviews</th>"
             "<th>Review Summary</th>"
             "</tr>")

    # Add a row for each restaurant
    for i, r in enumerate(restaurants):
        html += (f"<tr>"
                 f"<td>{i+1}</td>"
                 f"<td>{r.get('name','N/A')}</td>"
                 f"<td>{r.get('address','N/A')}</td>"
                 f"<td>{r.get('rating','N/A')}</td>"
                 f"<td>{r.get('reviews','N/A')}</td>"
                 f"<td>{r.get('review_summary','No summary available.')}</td>"
                 f"</tr>")

    # Close the table
    html += "</table>"
    return html

In [ ]:
# --- MAIN PROCESSING FUNCTION ---
def process_input(people_input, location_input, price_range_input, ambience_input, cuisines_input):
    """Process all inputs and return restaurant recommendations."""
    # --- STEP 1: VALIDATE AND PROCESS INPUTS ---

    # Check that location is provided
    user_location = location_input.strip()
    if not user_location:
        return "Please provide a location.", "", "", {}, ""

    # Add "Singapore" to the location if not already there
    if not user_location.endswith(", Singapore"):
        user_location += ", Singapore"

    # Split the people input into separate lines
    group_details = [line.strip() for line in people_input.splitlines() if line.strip()]
    if not group_details:
        return "Please provide at least one group detail.", "", "", {}, ""

    # Use either the provided cuisine input or extract from group details
    food_requirements = cuisines_input.strip() if cuisines_input.strip() else "; ".join(group_details)

    # Create a summary of all inputs
    combined_query_text = f"Location: {user_location}\nPrice Range: {price_range_input or 'Any'}\nAmbience: {ambience_input or 'Any'}\nFood Requirements: {food_requirements}"

    # --- STEP 2: FIND NEARBY RESTAURANTS ---
    logging.info(f"Fetching nearby restaurants for location: {user_location}...")
    nearby_restaurants = get_nearby_restaurants(user_location, GOOGLE_MAPS_API_KEY)

    # Check if any restaurants were found
    if not nearby_restaurants:
        return "No restaurants found nearby.", "", combined_query_text, {}, "No restaurants found."

    # --- STEP 3: FILTER RESTAURANTS BY QUALITY ---
    logging.info(f"Found {len(nearby_restaurants)} restaurants. Filtering by rating >= 3.9 and reviews > 20...")
    filtered_restaurants = []

    # Only keep restaurants with good ratings and enough reviews
    for r in nearby_restaurants:
        try:
            rating = float(r["rating"]) if r["rating"] != "N/A" else 0
            reviews = int(r["reviews"]) if r["reviews"] != "N/A" else 0
            if rating >= 3.9 and reviews > 20:
                filtered_restaurants.append(r)
        except Exception as e:
            continue

    # Check if any restaurants passed the filter
    if not filtered_restaurants:
        return "No restaurants meet the minimum review and rating criteria.", "", combined_query_text, {}, "No restaurants meet the criteria."

    # --- STEP 4: GET DETAILED RESTAURANT INFORMATION ---
    logging.info(f"{len(filtered_restaurants)} restaurants remain after filtering. Fetching detailed information...")

    # Save the location information for each restaurant
    original_geometry = {r["place_id"]: r.get("geometry", {}) for r in filtered_restaurants if r.get("place_id")}

    # Get only restaurants with valid place IDs
    valid_restaurants = [r for r in filtered_restaurants if r.get("place_id")]
    place_ids = [r["place_id"] for r in valid_restaurants]

    # Fetch detailed information for all restaurants at once
    try:
        detailed_restaurants = asyncio.run(fetch_all_restaurant_details(place_ids, GOOGLE_MAPS_API_KEY))
    except Exception as e:
        logging.error(f"Async error: {e}")
        detailed_restaurants = []

    # Add the location information back to the detailed results
    for dr in detailed_restaurants:
        pid = dr.get("place_id")
        if pid in original_geometry:
            dr["geometry"] = original_geometry[pid]

    logging.info(f"Fetched detailed info for {len(detailed_restaurants)} restaurants.")

    # --- STEP 5: GENERATE HTML TABLE ---
    table_html = generate_restaurants_table(detailed_restaurants)

    # --- STEP 6: CHOOSE THE BEST RESTAURANT ---
    chosen_restaurant, reasoning = choose_best_restaurant(
        detailed_restaurants, group_details, user_location, price_range_input, ambience_input, food_requirements
    )

    # --- STEP 7: PREPARE THE RECOMMENDATION ---
    if not chosen_restaurant:
        recommendation = "Error choosing the best restaurant."
    else:
        # Format the recommendation as Markdown text
        recommendation = f"**Recommendation:**\n\n{chosen_restaurant['name']} ({chosen_restaurant['address']})\n\n" \
                         f"Rating: {chosen_restaurant['rating']}, Total Reviews: {chosen_restaurant['reviews']}\n\n" \
                         f"Review Summary: {chosen_restaurant.get('review_summary','No review summary available.')}\n\n" \
                         f"**Reasoning:**\n{reasoning}"

    # --- STEP 8: GET MAP FOR CHOSEN RESTAURANT ---
    pid = chosen_restaurant.get("place_id") if chosen_restaurant else None
    geom = chosen_restaurant.get("geometry", {}) if chosen_restaurant else {}
    lat = geom.get("location", {}).get("lat")
    lng = geom.get("location", {}).get("lng")
    map_html = get_map_embed_html(pid, GOOGLE_MAPS_API_KEY) if pid else get_map_embed_html(None, GOOGLE_MAPS_API_KEY, lat, lng)

    # --- STEP 9: RETURN ALL RESULTS ---
    return recommendation, map_html, combined_query_text, detailed_restaurants, reasoning

In [ ]:
# Install necessary Python packages - these are libraries that add extra functionality
!pip install --upgrade openai requests aiohttp nest_asyncio gradio

# Import required libraries
import openai              # Used to communicate with AI models like ChatGPT
import json                # Helps work with JSON data (a common web data format)
import urllib.parse        # Helps format text for web addresses
import requests            # Allows the code to make web requests (like visiting websites)
import asyncio             # Helps run multiple tasks at the same time
import aiohttp             # Similar to requests but works with asyncio
import time                # Provides time-related functions
import logging             # Helps track what the program is doing
from functools import lru_cache  # Saves results of functions to avoid repeating work
import nest_asyncio        # Makes asyncio work in environments like Jupyter notebooks
import gradio as gr        # Creates simple web interfaces
from IPython.core.display import display, HTML  # Displays content in Jupyter notebooks

# Allow nested asyncio loops - technical setting needed for Jupyter/Colab
nest_asyncio.apply()

# Set up logging to track what's happening while the program runs
logging.basicConfig(level=logging.INFO)





# --- SET UP API KEYS ---
# You'll need to replace these with your own API keys
client = openai.OpenAI(api_key="YOUR_OPENAI_API_KEY")  # Connect to OpenAI (ChatGPT)
GOOGLE_MAPS_API_KEY = "YOUR_GOOGLE_MAPS_API_KEY"       # Connect to Google Maps









# --- LOCATION FINDER FUNCTION ---
# This function is "cached" - it remembers previous answers to avoid unnecessary work
@lru_cache(maxsize=32)
def geocode_location(location, maps_api_key):
    """Turn a location name into GPS coordinates (latitude and longitude)."""
    # Create a URL to ask Google Maps where this location is
    geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={urllib.parse.quote(location)}&key={maps_api_key}"

    try:
        # Send the request and get the response
        resp = requests.get(geocode_url)
        data = resp.json()  # Convert the response to JSON format

        # If Google found the location successfully
        if data["status"] == "OK":
            # Extract the latitude and longitude
            loc = data["results"][0]["geometry"]["location"]
            return loc["lat"], loc["lng"]
        else:
            # If there was an error, log it and return None
            logging.error(f"Geocode error for '{location}': {data.get('status')}")
            return None, None
    except Exception as e:
        # If something unexpected happened, log the error
        logging.error(f"Exception during geocoding: {e}")
        return None, None

# --- RESTAURANT SEARCH FUNCTION ---
def get_nearby_restaurants(location, maps_api_key):
    """Find restaurants within 1km of a location."""
    # First, convert the location name to coordinates
    lat, lng = geocode_location(location, maps_api_key)
    if lat is None or lng is None:
        return []  # Return empty list if we couldn't find the location

    # Create the URL for searching restaurants nearby
    base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=1000&type=restaurant&key={maps_api_key}"
    results = []
    next_page_token = None

    # Google might return multiple pages of results, so keep going until we get them all
    while True:
        # If this is a follow-up page, add the page token
        url = base_url if not next_page_token else f"{base_url}&pagetoken={next_page_token}"

        try:
            # Get the results for this page
            resp = requests.get(url)
            data = resp.json()

            # Check if the request was successful
            if data.get("status") not in ["OK", "ZERO_RESULTS"]:
                logging.error(f"Nearby search error: {data.get('status')}")
                break

            # Process each restaurant found
            for place in data.get("results", []):
                results.append({
                    "name": place.get("name", "Unknown"),           # Restaurant name
                    "address": place.get("vicinity", "Unknown"),    # Address
                    "place_id": place.get("place_id", None),        # Unique ID for this place
                    "rating": place.get("rating", "N/A"),           # Star rating
                    "reviews": place.get("user_ratings_total", "N/A"), # Number of reviews
                    "geometry": place.get("geometry", {})           # Location data
                })

            # Check if there are more pages of results
            next_page_token = data.get("next_page_token")
            if not next_page_token:
                break

            # Wait before requesting the next page (API requirement)
            time.sleep(2)

        except Exception as e:
            # Log any errors
            logging.error(f"Exception during Nearby Search: {e}")
            break

    return results

# --- DETAILED RESTAURANT INFO FUNCTION (ASYNC VERSION) ---
# This function gets more details about a restaurant and runs in the background
async def async_get_restaurant_details(session, place_id, maps_api_key):
    """Get detailed information about a specific restaurant."""
    # Create URL to get restaurant details
    url = (f"https://maps.googleapis.com/maps/api/place/details/json?"
           f"place_id={place_id}&fields=reviews,rating,user_ratings_total,formatted_address,name&key={maps_api_key}")

    try:
        # Get the detailed information
        async with session.get(url) as resp:
            data = await resp.json()

            # Set up default values in case some data is missing
            details = {
                "name": "Unknown",
                "address": "Unknown",
                "rating": "N/A",
                "reviews": "N/A",
                "review_summary": "No reviews found.",
                "place_id": place_id
            }

            # Extract the data if it exists
            if "result" in data:
                details["name"] = data["result"].get("name", "Unknown")
                details["address"] = data["result"].get("formatted_address", "Unknown")
                details["rating"] = data["result"].get("rating", "N/A")
                details["reviews"] = data["result"].get("user_ratings_total", "N/A")

                # If there are reviews, get the newest ones and analyze them
                if "reviews" in data["result"]:
                    sorted_reviews = sorted(data["result"]["reviews"], key=lambda x: x.get("time", 0), reverse=True)
                    newest_reviews = [rev.get("text", "") for rev in sorted_reviews[:5]]
                    if newest_reviews:
                        details["review_summary"] = analyze_reviews(newest_reviews)

            return details

    except Exception as e:
        # Log any errors
        logging.error(f"Error fetching details for place_id {place_id}: {e}")
        return {"place_id": place_id}

# This function gets details for multiple restaurants at once
async def fetch_all_restaurant_details(place_ids, maps_api_key):
    """Get details for multiple restaurants in parallel."""
    async with aiohttp.ClientSession() as session:
        # Create a task for each restaurant
        tasks = [async_get_restaurant_details(session, pid, maps_api_key) for pid in place_ids]
        # Run all tasks and wait for them to complete
        return await asyncio.gather(*tasks)

# --- REVIEW ANALYSIS FUNCTION ---
def analyze_reviews(reviews_list):
    """Use ChatGPT to analyze and summarize restaurant reviews."""
    # Combine all reviews into one text
    reviews_text = "\n".join(reviews_list)

    # Create a prompt for ChatGPT
    prompt = (
        "Analyze the following reviews for a restaurant and provide a concise summary "
        "that captures the overall sentiment and key points:\n\n"
        f"Reviews:\n{reviews_text}\n\nSummary:"
    )

    try:
        # Ask ChatGPT to analyze the reviews
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7  # Controls randomness: higher = more creative
        )
        # Get the summary from ChatGPT's response
        summary = response.choices[0].message.content.strip()
        return summary

    except Exception as e:
        # Log any errors
        logging.error(f"Error in ChatGPT review analysis: {e}")
        return "Review analysis failed."

# --- MAP EMBEDDING FUNCTION ---
def get_map_embed_html(place_id, maps_api_key, latitude=None, longitude=None):
    """Create HTML code to show a Google Map for a restaurant."""
    # If we have a place ID, use that (more accurate)
    if place_id:
        embed_url = f"https://www.google.com/maps/embed/v1/place?key={maps_api_key}&q=place_id:{place_id}"
    # Otherwise use the coordinates if available
    elif latitude and longitude:
        embed_url = f"https://www.google.com/maps/embed/v1/view?key={maps_api_key}&center={latitude},{longitude}&zoom=15"
    # If neither is available, show an error message
    else:
        return "<p>No map available.</p>"

    # Return an HTML iframe that shows the map
    return f'<iframe width="100%" height="400" frameborder="0" style="border:0" src="{embed_url}" allowfullscreen></iframe>'

# --- RESTAURANT RECOMMENDATION FUNCTION ---
def choose_best_restaurant(restaurants, people_details, location, price_range, ambience, food_requirements):
    """Use ChatGPT to choose the best restaurant for a group of people."""
    # Start building a prompt for ChatGPT
    prompt = f"Given the following list of restaurants near {location} in Singapore and these group details:\n"

    # Add each person's requirements
    for detail in people_details:
        prompt += f"- {detail}\n"

    # Add other preferences
    prompt += f"Price Range: {price_range or 'Any'}\n"
    prompt += f"Ambience: {ambience or 'Any'}\n"
    prompt += f"Food Requirements: {food_requirements or 'None'}\n\n"

    # Add instructions for ChatGPT
    prompt += ("For each restaurant below, note its name, address, rating, total reviews, and review summary. "
               "Then, choose the single best restaurant that best meets the group's needs that do not have serious negative reviews, with higher priority on fulfilling each individual's specific requirements. "
               "In your explanation, you MUST clearly state how the restaurant meets every individual's needs cannot just group them together and say it is good for the group. "
               "If any requirement cannot be met, explain why a compromise was necessary and include an apology for that compromise. "
               "Use the restaurant's number (e.g., '1', '2', etc.) in your response.\n\n")

    # Add information about each restaurant
    for i, r in enumerate(restaurants):
        prompt += (f"{i+1}. {r['name']} ({r['address']}) - Rating: {r['rating']}, Reviews: {r['reviews']}\n"
                   f"   Review Summary: {r.get('review_summary', 'No summary available.')}\n\n")

    # Ask for a structured response
    prompt += "Output JSON with keys 'chosen_restaurant_index' and 'reason':"

    try:
        # Ask ChatGPT to choose the best restaurant
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )

        # Extract and process the response
        content = response.choices[0].message.content.strip()
        chosen = json.loads(content)  # Convert the JSON response to a Python object

        # Get the chosen restaurant index (subtract 1 because Python uses 0-based indexing)
        index = int(chosen.get("chosen_restaurant_index")) - 1

        # Make sure the index is valid
        if index < 0 or index >= len(restaurants):
            logging.error("Chosen index out of range. Defaulting to the first restaurant.")
            index = 0

        # Get the reasoning
        reason = chosen.get("reason", "No reasoning provided.")

        # Return the chosen restaurant and the reasoning
        return restaurants[index], reason

    except Exception as e:
        # Log any errors
        logging.error(f"Error in choosing best restaurant: {e}")
        return None, f"Error: {e}"

# --- HTML TABLE GENERATOR ---
def generate_restaurants_table(restaurants):
    """Create an HTML table showing all restaurant details."""
    # Start the HTML table
    html = "<table border='1' style='border-collapse: collapse; width: 100%;'>"

    # Add table headers
    html += ("<tr>"
             "<th>#</th>"
             "<th>Name</th>"
             "<th>Address</th>"
             "<th>Rating</th>"
             "<th>Total Reviews</th>"
             "<th>Review Summary</th>"
             "</tr>")

    # Add a row for each restaurant
    for i, r in enumerate(restaurants):
        html += (f"<tr>"
                 f"<td>{i+1}</td>"
                 f"<td>{r.get('name','N/A')}</td>"
                 f"<td>{r.get('address','N/A')}</td>"
                 f"<td>{r.get('rating','N/A')}</td>"
                 f"<td>{r.get('reviews','N/A')}</td>"
                 f"<td>{r.get('review_summary','No summary available.')}</td>"
                 f"</tr>")

    # Close the table
    html += "</table>"
    return html

# --- MAIN PROCESSING FUNCTION ---
def process_input(people_input, location_input, price_range_input, ambience_input, cuisines_input):
    """Process all inputs and return restaurant recommendations."""
    # --- STEP 1: VALIDATE AND PROCESS INPUTS ---

    # Check that location is provided
    user_location = location_input.strip()
    if not user_location:
        return "Please provide a location.", "", "", {}, ""

    # Add "Singapore" to the location if not already there
    if not user_location.endswith(", Singapore"):
        user_location += ", Singapore"

    # Split the people input into separate lines
    group_details = [line.strip() for line in people_input.splitlines() if line.strip()]
    if not group_details:
        return "Please provide at least one group detail.", "", "", {}, ""

    # Use either the provided cuisine input or extract from group details
    food_requirements = cuisines_input.strip() if cuisines_input.strip() else "; ".join(group_details)

    # Create a summary of all inputs
    combined_query_text = f"Location: {user_location}\nPrice Range: {price_range_input or 'Any'}\nAmbience: {ambience_input or 'Any'}\nFood Requirements: {food_requirements}"

    # --- STEP 2: FIND NEARBY RESTAURANTS ---
    logging.info(f"Fetching nearby restaurants for location: {user_location}...")
    nearby_restaurants = get_nearby_restaurants(user_location, GOOGLE_MAPS_API_KEY)

    # Check if any restaurants were found
    if not nearby_restaurants:
        return "No restaurants found nearby.", "", combined_query_text, {}, "No restaurants found."

    # --- STEP 3: FILTER RESTAURANTS BY QUALITY ---
    logging.info(f"Found {len(nearby_restaurants)} restaurants. Filtering by rating >= 3.9 and reviews > 20...")
    filtered_restaurants = []

    # Only keep restaurants with good ratings and enough reviews
    for r in nearby_restaurants:
        try:
            rating = float(r["rating"]) if r["rating"] != "N/A" else 0
            reviews = int(r["reviews"]) if r["reviews"] != "N/A" else 0
            if rating >= 3.9 and reviews > 20:
                filtered_restaurants.append(r)
        except Exception as e:
            continue

    # Check if any restaurants passed the filter
    if not filtered_restaurants:
        return "No restaurants meet the minimum review and rating criteria.", "", combined_query_text, {}, "No restaurants meet the criteria."

    # --- STEP 4: GET DETAILED RESTAURANT INFORMATION ---
    logging.info(f"{len(filtered_restaurants)} restaurants remain after filtering. Fetching detailed information...")

    # Save the location information for each restaurant
    original_geometry = {r["place_id"]: r.get("geometry", {}) for r in filtered_restaurants if r.get("place_id")}

    # Get only restaurants with valid place IDs
    valid_restaurants = [r for r in filtered_restaurants if r.get("place_id")]
    place_ids = [r["place_id"] for r in valid_restaurants]

    # Fetch detailed information for all restaurants at once
    try:
        detailed_restaurants = asyncio.run(fetch_all_restaurant_details(place_ids, GOOGLE_MAPS_API_KEY))
    except Exception as e:
        logging.error(f"Async error: {e}")
        detailed_restaurants = []

    # Add the location information back to the detailed results
    for dr in detailed_restaurants:
        pid = dr.get("place_id")
        if pid in original_geometry:
            dr["geometry"] = original_geometry[pid]

    logging.info(f"Fetched detailed info for {len(detailed_restaurants)} restaurants.")

    # --- STEP 5: GENERATE HTML TABLE ---
    table_html = generate_restaurants_table(detailed_restaurants)

    # --- STEP 6: CHOOSE THE BEST RESTAURANT ---
    chosen_restaurant, reasoning = choose_best_restaurant(
        detailed_restaurants, group_details, user_location, price_range_input, ambience_input, food_requirements
    )

    # --- STEP 7: PREPARE THE RECOMMENDATION ---
    if not chosen_restaurant:
        recommendation = "Error choosing the best restaurant."
    else:
        # Format the recommendation as Markdown text
        recommendation = f"**Recommendation:**\n\n{chosen_restaurant['name']} ({chosen_restaurant['address']})\n\n" \
                         f"Rating: {chosen_restaurant['rating']}, Total Reviews: {chosen_restaurant['reviews']}\n\n" \
                         f"Review Summary: {chosen_restaurant.get('review_summary','No review summary available.')}\n\n" \
                         f"**Reasoning:**\n{reasoning}"

    # --- STEP 8: GET MAP FOR CHOSEN RESTAURANT ---
    pid = chosen_restaurant.get("place_id") if chosen_restaurant else None
    geom = chosen_restaurant.get("geometry", {}) if chosen_restaurant else {}
    lat = geom.get("location", {}).get("lat")
    lng = geom.get("location", {}).get("lng")
    map_html = get_map_embed_html(pid, GOOGLE_MAPS_API_KEY) if pid else get_map_embed_html(None, GOOGLE_MAPS_API_KEY, lat, lng)

    # --- STEP 9: RETURN ALL RESULTS ---
    return recommendation, map_html, combined_query_text, detailed_restaurants, reasoning

# --- WEB INTERFACE SETUP ---
# Create a Gradio web interface to use this code
with gr.Blocks() as demo:
    # Add a header
    gr.Markdown("## What to eat?")

    # Create a two-column layout
    with gr.Row():
        # Left column: Input fields
        with gr.Column():
            # Text field for group requirements
            people_input = gr.Textbox(
                label="People Details",
                placeholder="Enter one person's detail per line (e.g. 'Tom wants aircon place', 'Sam wants spicy food', 'Ben wants cheap food')",
                lines=5
            )
            # Text field for location
            location_input = gr.Textbox(
                label="Location",
                placeholder="e.g. Pioneer"
            )
            # Text field for price range
            price_range_input = gr.Textbox(
                label="Price Range (optional)",
                placeholder="e.g. Low, Medium, High"
            )
            # Text field for ambience
            ambience_input = gr.Textbox(
                label="Ambience (optional)",
                placeholder="e.g. cosy, modern, casual"
            )
            # Text field for specific cuisines
            cuisines_input = gr.Textbox(
                label="Specific Cuisines / Food Requirements (optional)",
                placeholder="e.g. Thai, Mexican, Indian, or custom requirements"
            )
            # Submit button
            submit_button = gr.Button("Get Recommendation")

        # Right column: Results
        with gr.Column():
            # Display for the final recommendation
            recommendation_output = gr.Markdown(label="ChatGPT Recommendation")
            # Display for the map
            map_output = gr.HTML(label="Map View")
            # Display for the query summary
            combined_query_output = gr.Textbox(label="Combined Query Info")
            # Display for detailed restaurant list
            detailed_restaurants_output = gr.JSON(label="Detailed Restaurant List with Review Summaries")
            # Display for ChatGPT's reasoning
            chatgpt_reason_output = gr.Markdown(label="ChatGPT Reasoning and Explanation")

    # Connect the submit button to the processing function
    submit_button.click(
        fn=process_input,
        inputs=[people_input, location_input, price_range_input, ambience_input, cuisines_input],
        outputs=[recommendation_output, map_output, combined_query_output, detailed_restaurants_output, chatgpt_reason_output]
    )

# Launch the web interface
demo.launch(share=True, inline=True)